# Stochastic Gradient Descent from Scratch

This notebook expands the upon the [*Gradient Descent from Scratch* notebook](https://github.com/jonkrohn/ML-foundations/blob/master/notebooks/gradient-descent-from-scratch.ipynb) to introduce *stochastic* gradient descent (SGD).